In [14]:
import cv2
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from sklearn import cluster


In [19]:
import os
filename = []
image_path = './Dataset/'
save_path ='./Data/'
files= os.listdir(image_path)
for file in files:
    if file.find('gt') == -1:      
        filename.append(file)
print(filename)       
#filenamee = '33.bmp'
#file_path = os.path.join(image_path, filenamee)

['6146_4739-4005_5986-10334_4917-25.bmp', '33.bmp', '7614_6285-4139_10751-8597_6032-21.bmp']


In [21]:
def prepossing_img(image_path,file, save_path):
    fileinfo = re.split(r"[-,.]",file)
    filenum = fileinfo[-2]
    file_path = os.path.join(image_path, file)
    img = cv2.imread(file_path,1)
    data = []
    bbox = []
    
    ret,thresh1=cv2.threshold(img,110,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((3,3),np.uint8)  
    dilation = cv2.dilate(thresh1,kernel,iterations =2)
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    edge = cv2.Canny(dilation, 125, 255) 
    gray = cv2.cvtColor(dilation,cv2.COLOR_BGR2GRAY)
    imge,contours, hierarchy = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for i in contours:

        x,y,w,h=cv2.boundingRect(i)
        data.append([x+w/2,y+h/2])
        bbox.append([w,h])
        
    np.savez(save_path+str(filenum)+'-box'+".npz", data=data,box=bbox)


In [22]:
for file in filename:
    prepossing_img(image_path,file, save_path)
print('preprocessing is done')
    

preprocessing is done


In [ ]:
# In[14]:


data=np.array(data)
print(data.shape)
labels = range(1, 142)  
plt.figure(figsize=(10, 7))  
plt.subplots_adjust(bottom=0.1)  
plt.scatter(data[:,0],data[:,1], label='True Position')

for label, x, y in zip(labels, data[:, 0], data[:, 1]):  
    plt.annotate(
        label,
        xy=(x, y), xytext=(-3, 3),
        textcoords='offset points', ha='right', va='bottom')
plt.show()  

In [ ]:
# In[15]:


from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import time
start = time.time()
linked = linkage(data, 'average')
labelList = range(1, np.array(data).shape[0]+1)

plt.figure(figsize=(10, 7))  
dendrogram(linked,  
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()  
c, coph_dists = cophenet(linked, pdist(data))
print (c)
# print(linked[0])
print('end',time.time()-start)

In [ ]:
# In[16]:



from scipy.cluster.hierarchy import fcluster
max_d = 3000
clusters = fcluster(linked, max_d, criterion='distance')
C=[]
W=[]
for i in range(max(clusters)):
    
    index = np.where(clusters==i+1)
    points = np.take(data,index,0)
    BB=points[0,:,:]
    #print(BB)
    mean=np.mean(BB,0)
    print(mean)
    C.append(mean)
    W.append(np.sqrt(np.var(BB,0)))
    print('var',np.sqrt(np.var(BB,0)))
C=np.array(C)
j=np.array([3600,3600])
imga = cv2.imread('33.bmp',1)

for i in C:
    print('i',int(i[0]),int(i[1]))
#     for j in centroids_b:
    cv2.rectangle(imga, (int(i[0])-j[0]/2, int(i[1])-j[1]/2), (int(i[0]) + j[0]/2, int(i[1]) + j[1]/2), (255, 0, 0), 8)
    #cv2.rectangle(imga, (50, 50), (5000, 5000), (222, 0, 0),5)
    cv2.circle(imga,(int(i[0]),int(i[1])),100,(55,255,155),-1)#修改最后一个参数
plt.figure(figsize=(30,30))
plt.imshow(imga)
    
    
    

In [ ]:
# In[17]:


import matplotlib.pyplot as plt  
get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np  
shape = data.shape  
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=3,affinity='l2', linkage='average',compute_full_tree='auto')  
A=cluster.fit_predict(data)  
plt.figure(figsize=(10, 7))  
plt.scatter(data[:,0], data[:,1], c=cluster.labels_, cmap='rainbow')  

In [ ]:
# In[23]:


print(A)
cluster.n_clusters
center = []
for i in range(cluster.n_clusters):
    index = np.where(A==i)
    points = np.take(data,index,0)
    BB=points[0,:,:]
    #print(BB)
    mean=np.mean(BB,0)
    print(mean)
    center.append(mean)
    print(np.sqrt(np.var(BB,0)))
    

In [ ]:
# In[24]:


print(center)
j=np.array([336,336])
for i in center:
#     for j in centroids_b:
    cv2.rectangle(img, (int(i[0])-int(j[0]/2), int(i[1])-int(j[1])/2), (int(i[0]) + int(j[0])/2, int(i[1]) + int(j[1])/2), (222, 0, 0), 1)
plt.figure(figsize=(20,20))
plt.imshow(a)

In [ ]:
# In[72]:


estimator=KMeans(n_clusters=4)
res=estimator.fit_predict(data)
lable_pred=estimator.labels_
centroids=estimator.cluster_centers_
inertia=estimator.inertia_
print (lable_pred)
print (centroids)
print (inertia)

# estimator_b=KMeans(n_clusters=3)
# res_b=estimator_b.fit_predict(data)
# lable_pred_b=estimator_b.labels_
# centroids_b=estimator_b.cluster_centers_
# inertia_b=estimator_b.inertia_
# print (lable_pred_b)
# print (centroids_b)
# print (inertia_b)


# In[49]:


centroids_b=np.array([[224,224],[336,336]])
print(centroids_b.shape)
for i in centroids:
    for j in centroids_b:
        cv2.rectangle(img, (int(i[0])-int(j[0]/2), int(i[1])-int(j[1])/2), 
                      (int(i[0]) + int(j[0])/2, int(i[1]) + int(j[1])/2), (222, 111, 0), 1)
plt.figure(figsize=(20,20))
plt.imshow(a)